In [19]:
import pandas as pd
import random
base = 'database.txt'

data = pd.read_csv(base, sep='\t', header=None)

In [20]:
IMPORTANT = 1
ABSTAIN = 0

In [21]:
data

,0
0,'@galliyanamiteva kkkkkkkkkk eu sou bue caseir...
1,"'Covid-19: Brasil tem 28,7 milhões de casos e ..."
2,'A mesma globo que recentemente defendia o iso...
3,'Covid era altura de fazeres a tua parte e aju...
4,'Feriadão prolongado desse e não vou poder ver...
...,...
5000095,"'@Pequenaquenga É tipo o teste do covid, fazia..."
5000096,'#Cloroquina | Ex-ministro Mandetta reafirmou ...
5000097,'covid vai embora nunca?'
5000098,'Dia até agora constrangedor na CPI. Senadores...


In [22]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(data,train_size=0.8)

In [23]:
df_train

,0
3064391,'Auxílio de R$ 500 para órfãos da pandemia da ...
2689554,'50 mil doses de vacina contra a Covid-19 são ...
551327,"'Bebê leva, por engano, seis doses de vacina c..."
2385985,'A pessoa posta vídeo no fervo e me chama pra ...
2481171,"'Para ler a edição na íntegra, acesse: https:/..."
...,...
2447081,'Tentando dar uma finalizada no Batman Arkham ...
642603,'Tô com covid :/\n\nPor enquanto ainda bem só ...
1794899,'@o_antagonista Devem estar com medo que a vov...
4176519,'@mu_candido @drbluecake @pedromvmartins @gabr...


In [24]:
from snorkel.labeling import labeling_function,PandasLFApplier,LFAnalysis

In [25]:
@labeling_function()
def label1(x):
    return IMPORTANT if "vacina" in x.text.lower() else ABSTAIN


@labeling_function()
def label2(x):
    return IMPORTANT if "bolsonaro" in x.text.lower() else ABSTAIN


@labeling_function()
def label3(x):
    return IMPORTANT if "pandemia" in x.text.lower() else ABSTAIN

In [26]:
lfs = [label1, label2, label3]

In [27]:
applier = PandasLFApplier(lfs=lfs)

In [28]:
applier

PandasLFApplier, LFs: ['label1', 'label2', 'label3']

In [29]:
L_train = applier.apply(df=df_train)

  0%|          | 1/4000080 [00:00<37:47, 1763.79it/s]


AttributeError: 'Series' object has no attribute 'text'

In [ ]:
L_train

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 0]])

In [ ]:
LFAnalysis(L=L_train,lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
label1,0,"[0, 1]",1.0,1.0,0.38921
label2,1,"[0, 1]",1.0,1.0,0.38921
label3,2,"[0, 1]",1.0,1.0,0.38921


In [ ]:
from snorkel.labeling.model import MajorityLabelVoter,LabelModel

In [ ]:
label_model = LabelModel(cardinality=2,verbose=True)
label_model.fit(L_train=L_train,n_epochs=500,log_freq=100,seed=123)

INFO:root:Computing O...
INFO:root:Estimating \mu...
  0%|          | 0/500 [00:00<?, ?epoch/s]INFO:root:[0 epochs]: TRAIN:[loss=3.388]
INFO:root:[100 epochs]: TRAIN:[loss=0.003]
 35%|███▌      | 175/500 [00:00<00:00, 1743.76epoch/s]INFO:root:[200 epochs]: TRAIN:[loss=0.001]
INFO:root:[300 epochs]: TRAIN:[loss=0.001]
100%|██████████| 500/500 [00:00<00:00, 1806.02epoch/s]
INFO:root:Finished Training


In [ ]:
df_train.head()

,is_retweet,text
1449,False,If you treat people right they will treat you ...
25120,False,They just arrested pol Shelly Silver in New Yo...
32317,False,ObamaCare will explode and we will all get tog...
21914,False,"""@BettyeBear: @realDonaldTrump Every working c..."
1610,False,Aberdeen tourism is booming because of my grea...


In [ ]:
df_train['Labels'] = label_model.predict(L=L_train,tie_break_policy="abstain")

/var/folders/47/2r15btws4hl473544pb4_s700000gp/T/ipykernel_65770/1145111523.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['Labels'] = label_model.predict(L=L_train,tie_break_policy="abstain")


In [ ]:
df_train.query('Labels == 0')

,is_retweet,text,Labels
1449,False,If you treat people right they will treat you ...,0
25120,False,They just arrested pol Shelly Silver in New Yo...,0
32317,False,ObamaCare will explode and we will all get tog...,0
21914,False,"""@BettyeBear: @realDonaldTrump Every working c...",0
1610,False,Aberdeen tourism is booming because of my grea...,0
...,...,...,...
23766,False,"""@gabriellebragg4: How do I find out where @re...",0
7763,False,.@TrumpDoral’s record $200M renovations are on...,0
15377,False,"""@TheBigJamesG staff at Trump Doral was amazi...",0
17730,False,"""@pandoerra: @realDonaldTrump I think you coul...",0
